# Jupyter Notebook approach for SCORE interaction


### Necessary packages 

In [1]:
from checkscore.methods import readonly, external, payable
from checkscore.methods import Checkscore
from iconsdk.wallet.wallet import KeyWallet

password = input()
deployer_wallet = KeyWallet.load("keystore_ibriz.json",password)
deployer_wallet.get_address()

SCORE_ADDRESS = ""
Checkscore(SCORE_ADDRESS).fill_methods()

ModuleNotFoundError: No module named 'checkscore'

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [233]:
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
import requests
import json
import pickle
import csv
import os
from pprint import pprint

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

mainnet= {
    "iconservice":"https://ctz.solidwallet.io",
    "nid" :1
    
}

yeouido={
     "iconservice":"https://bicon.net.solidwallet.io",
     "nid" : 3
    
}
euljiro={
     "iconservice":"https://test-ctz.solidwallet.io",
     "nid" : 2
    
}
pagoda={
    "iconservice":"https://zicon.net.solidwallet.io",
     "nid" :80
    
    
}
custom={
    "iconservice":"http://18.237.205.52:9000/",
     "nid":3
    
    
}


### Required service

In [3]:
env = custom #mainnet,yeouido,euljiro,pagoda,custom
icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]



### A list for SCORE addresses 

In [4]:
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"

# Yeouido Testnet
# SCORE_ADDRESS = "cxf42190e91701bec907c7c4514eda6ee0af7f795b"
SCORE_ADDRESS = "cx19f99666108bab3db87d1c633c2bfb8d0b219433"


# Mainnet}
# SCORE_ADDRESS = "cx346a2c30ea2712557deb4d7392688750350d1dea"

### Importing necessary wallets

If you want to use your pre-exisiting wallets, execute the cell below:-

In [5]:
password = input()

deployer_wallet = KeyWallet.load("keystore_2",password)
deployer_wallet.get_address()


Liverpool@1


'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'

### OR creating new wallets 

If you want to use new wallets, execute the cell below:-

In [ ]:
deployer_wallet = KeyWallet.create()
print(deployer_wallet.get_address())
caller_wallet = KeyWallet.create()
print(caller_wallet.get_address())

### Loading ICX
Go to [ibriz faucet](https://icon-faucet.ibriz.ai/) to load ICX into your wallets.

### Deploying contract

In [113]:
# params = {'_initialSupply':500000000, '_decimals': 18}
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxad29953f5b8957704e9092632bccc3b1aac62eea")\
    .nid(3)\
    .nonce(100)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('addressProvider'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'txHash': '0xbac627096d270e8aab79e8cb88d894153b0dc927870000887e18e7016f3b60c1',
 'blockHeight': 100061,
 'blockHash': '0x966f7abe16fe3c2c2e4b94653374fd95b23c09140b70095907314fc353b83058',
 'txIndex': 1,
 'to': 'cxad29953f5b8957704e9092632bccc3b1aac62eea',
 'scoreAddress': 'cxad29953f5b8957704e9092632bccc3b1aac62eea',
 'stepUsed': 1631208800,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1631208800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

### Updating contract

In [236]:
deploy_transaction = DeployTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c")\
    .nid(NID)\
    .nonce(1000)\
    .content_type("application/zip")\
    .content(gen_deploy_data_content('lendingPoolCore'))\
    .build()

estimate_step = icon_service.estimate_step(deploy_transaction)
step_limit = estimate_step + 100000
signed_transaction = SignedTransaction(deploy_transaction, deployer_wallet, step_limit)

tx_hash = icon_service.send_transaction(signed_transaction)


get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x93648a2bae562baddf4f08496f105ca79f344a0fdca30ae40bac14ffba80f5ab',
 'blockHeight': 108020,
 'blockHash': '0x00460e9592e15ce25a0c60d54cca74cd96d1a422e746f5ac8867ab26331effc4',
 'txIndex': 1,
 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
 'scoreAddress': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
 'stepUsed': 1805127200,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 1805127200,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Adding new reserve in LendingPoolCore



In [ ]:
params ={"_reserve": {"reserveAddress":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","oTokenAddress":"cx48dd26868c25cfa744aab9f2fccd76ed40c41454","totalBorrows":"0","lastUpdateTimestamp": "0","liquidityRate":"0","borrowRate":"0","liquidityCumulativeIndex":f"1{'0'*18}","borrowCumulativeIndex":f"1{'0'*18}","baseLTVasCollateral":"60","liquidationThreshold":"65","liquidationBonus":"10","decimals":"18","borrowingEnabled": "1","usageAsCollateralEnabled":"1","isFreezed":"0","isActive":"1"} }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("addReserveData")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

## Setting reserve constants in LendingPoolCore

In [8]:
params ={"_constants": [{"reserve":"cx072d8f95877a53df350e3dc3d8dba2f379037d42","optimalUtilizationRate":f"6{'0'*17}","baseBorrowRate":"0","slopeRate1":f"7{'0'*16}","slopeRate2":f"3{'0'*18}"} ]}
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("setReserveConstants")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c', 'stepLimit': '0x45178', 'timestamp': '0x5b308f33b3bd9', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setReserveConstants', 'params': {'_constants': [{'reserve': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42', 'optimalUtilizationRate': '600000000000000000', 'baseBorrowRate': '0', 'slopeRate1': '70000000000000000', 'slopeRate2': '3000000000000000000'}]}}, 'signature': 'J1RWKTbkS02BgRN4jZH6LrSP2ympGg/bmxdGFV9G2sM/C9CZn3CX5aunvQKh2DmupJM6LxO72corUhrQ1VFfLAA='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x5ba2b5b85cc0f9a9d25a3108984d4e7fffa4f106d358df17e93cd0ad8bbb0518',
 'blockHeight': 95127,
 'blockHash': '0x6baea1a60b2dc04d17a42eba1e6c144dfa4219faf70b8372bcd40fa637ddf2dc',
 'txIndex': 1,
 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
 'stepUsed': 183000,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 183000,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Setting interfaces in LendingPool

In [10]:
params ={"_address": "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxbf1a257dcb378b0fd41066fa87abef3f25992832") \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolCoreAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832', 'stepLimit': '0x3db30', 'timestamp': '0x5b30902f19fad', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setLendingPoolCoreAddress', 'params': {'_address': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c'}}, 'signature': '9FBOBTy4THd08vYX10xdGDs7kCmf3A++5OrFGxMPsIliQJQkOOirZ7EY01llX27DV+lOY8Y21skkp1GublMWUwE='}


{'txHash': '0x5ca1ed7362148f9b117259ef94d858e6402732d2fdf9b26bc7aabde69477fc96',
 'blockHeight': 95258,
 'blockHash': '0xedf8170764a54de4541ceb21875022bedecfdd2433bd757018d36145c020fd49',
 'txIndex': 1,
 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
 'stepUsed': 152720,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 152720,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [12]:
params ={"_address": "cx072d8f95877a53df350e3dc3d8dba2f379037d42" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxbf1a257dcb378b0fd41066fa87abef3f25992832") \
    .nid(NID) \
    .nonce(100) \
    .method("setUSDbAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832', 'stepLimit': '0x3d298', 'timestamp': '0x5b309044d16af', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setUSDbAddress', 'params': {'_address': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42'}}, 'signature': 'QudO9QJkDPtvc5UFbQIgBvjDMc/Vj7RJ5zlKcljU6PU9qKdNNaXZIVLeuAWsCCoNGLaFg88mm+pt3L7QhVeDFgE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x6d25b73a67de8cc1c15422d1bd91b24d987fd7049fe1c2ff095231082048ef2b',
 'blockHeight': 95270,
 'blockHash': '0xa6c15d3ae47aeac3a6d0159aebb685bb1cadad28aee31bf411631542e1f621db',
 'txIndex': 1,
 'to': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
 'stepUsed': 150520,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 150520,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

## Setting interfaces in LendingPoolDataProvider

In [14]:
params ={"_address": "cx162fe5b5d3276e9493de1fd440511f0844ca9a7c" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("setLendingPoolCoreAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx84490f03cc04b799fe668545340eaeac7cae628c', 'stepLimit': '0x3db30', 'timestamp': '0x5b30905a139f8', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setLendingPoolCoreAddress', 'params': {'_address': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c'}}, 'signature': 'sATOVxx94uGnvFWux4iUTuPReQMw4M1yVjPo1GBLSdxH+m8AQwftr5gJh86bX5sD7T1/Lh2FlIL/YBPj94USdAE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x78af14823370c3c75204802d846d3f9cf849e8016b741b03e0c953d0b75b14b2',
 'blockHeight': 95281,
 'blockHash': '0xdb46541a96584cfcaba5ac4a106349cf6eedf765561f2f824721b37b25e4ac83',
 'txIndex': 1,
 'to': 'cx84490f03cc04b799fe668545340eaeac7cae628c',
 'stepUsed': 152720,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 152720,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [15]:
params ={"_address": "cx293582520a75648d91a3c454ad9657aa4cbf1f7d" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c") \
    .nid(NID) \
    .nonce(100) \
    .method("setOracleAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)


{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx84490f03cc04b799fe668545340eaeac7cae628c', 'stepLimit': '0x3d428', 'timestamp': '0x5b30906951406', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setOracleAddress', 'params': {'_address': 'cx293582520a75648d91a3c454ad9657aa4cbf1f7d'}}, 'signature': 'v4cU6qZb3shonfTy/1wZHgeAyWKfqj2Ynv91VJMOwzoIL6zOdtlAnkotsc4zIlBWvz7CbMsweBL8RN+A/L0sSgA='}


## Setting Price in Oracle

In [140]:
params ={"_address": "cx072d8f95877a53df350e3dc3d8dba2f379037d42" }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx48dd26868c25cfa744aab9f2fccd76ed40c41454") \
    .nid(NID) \
    .nonce(100) \
    .method("setReserveAddress")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454', 'stepLimit': '0x3d4f0', 'timestamp': '0x5b30ba5edca95', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'setReserveAddress', 'params': {'_address': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42'}}, 'signature': 'Poi/fs9sSuF6qw3A4xTVcj2a4iH8ntam9pNvCCrIlp4YUHe+Pjp2q89eeb+Wf8B3kN51NzGBwPuhb31OPkKGoAA='}


In [198]:
params ={"_reserveAddress": "cx072d8f95877a53df350e3dc3d8dba2f379037d42","_liquidityRate":10**18 }
call_transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c") \
    .nid(NID) \
    .nonce(100) \
    .method("updateLiquidityRate")\
    .params(params)\
    .build()

estimate_step = icon_service.estimate_step(call_transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(call_transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

{'version': '0x3', 'from': 'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce', 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c', 'stepLimit': '0x3e120', 'timestamp': '0x5b30e4f2ab800', 'nid': '0x3', 'nonce': '0x64', 'dataType': 'call', 'data': {'method': 'updateLiquidityRate', 'params': {'_reserveAddress': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42', '_liquidityRate': '0xde0b6b3a7640000'}}, 'signature': 'Oe7n5oCIiWpsDth5w79oQrf2nbJLK+peRnetl2h/Y89w468ky9kpmm+N2FIBUp1Z+Kn2lc9kuIq1F8XyQAmKDQE='}


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0x89c7addd1655acc6de09cbd9d987718f52538d8106ef0cdaf34076e83ce31a92',
 'blockHeight': 106621,
 'blockHash': '0x4ffb3629338d24aae101715377d7f0fb1bbdb68ad16a7deea1a93617371e6049',
 'txIndex': 1,
 'to': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
 'stepUsed': 154240,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 154240,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x

## Depositing USDb

In [231]:

data = "{\"method\": \"deposit\", \"params\": {\"amount\": 1000000000000000000000}}".encode("utf-8")
params = {"_to": "cxbf1a257dcb378b0fd41066fa87abef3f25992832", "_value": "1000000000000000000000", "_data": data}
transaction = CallTransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx072d8f95877a53df350e3dc3d8dba2f379037d42")\
    .value(0)\
    .step_limit(10000000)\
    .nid(3)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, deployer_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
get_tx_result(tx_hash)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


{'code': -32602, 'message': 'Pending transaction'} (Json rpc error), Retrying in 1 seconds...


{'txHash': '0xe2f5dd28dbeee3b47edb74900015d2da4412d0deaf7c90a318a139b956055870',
 'blockHeight': 107813,
 'blockHash': '0x6c3dc233e57b1a5f18920a2b5772aa72e5d8312656e82b059d7ca4f8b61beb8c',
 'txIndex': 1,
 'to': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
 'stepUsed': 737440,
 'stepPrice': 10000000000,
 'cumulativeStepUsed': 737440,
 'eventLogs': [{'scoreAddress': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832',
   'indexed': ['Test(str)'],
   'data': ['{"method": "deposit", "params": {"amount": 1000000000000000000000}}']},
  {'scoreAddress': 'cx162fe5b5d3276e9493de1fd440511f0844ca9a7c',
   'indexed': ['ReserveUpdated(Address,int,int,int,int)',
    'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
    '0x0',
    '0x0'],
   'data': ['0xde0b6b3a7640000', '0xde0b6b3a7640000']},
  {'scoreAddress': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454',
   'indexed': ['Mint(Address,int)',
    'hx91bf040426f226b3bfcd2f0b5967bbb0320525ce'],
   'data': ['0x0']},
  {'scoreAddress': 'cx48dd26868c25cfa744aab9f2f

### ICX Transfer

In [ ]:
transaction = TransactionBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("hx233633ad3d85fe2aed5ac71863ac5d7a2f31d16b")\
    .nid(3) \
    .nonce(100) \
    .value(10000*10**18)\
    .build()

estimate_step = icon_service.estimate_step(transaction)
step_limit = estimate_step + 100000
# Returns the signed transaction object having a signature
signed_transaction = SignedTransaction(transaction, deployer_wallet,step_limit)

# Reads params to transfer to nodes
print(signed_transaction.signed_transaction_dict)

# Sends the transaction
tx_hash = icon_service.send_transaction(signed_transaction)

get_tx_result(tx_hash)

In [ ]:
balance = icon_service.get_balance("hxedcdfa97532eb690cf65c0f27ca1b5d4374b8122")
print(balance)

In [86]:
# params = {'_address': 'hxc9319d218ffff1672c9f1c15365c6f6c0d161c7b' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cxad29953f5b8957704e9092632bccc3b1aac62eea")\
    .method("getAllAddress")\
    .build()
response = icon_service.call(_call)
print(response)

{'collateral': {'USDb': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42', 'sICX': ''}, 'oTokens': {'oUSDb': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454', 'oICX': ''}, 'systemContract': {'LendingPool': 'cxbf1a257dcb378b0fd41066fa87abef3f25992832', 'LendingPoolDataProvider': 'cx84490f03cc04b799fe668545340eaeac7cae628c'}}


In [237]:
params = {'_reserve': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c")\
    .method("getReserveData")\
    .params(params)\
    .build()
response = icon_service.call(_call)
pprint(response)

{'availableLiquidity': '0x3635c9adc5dea00000',
 'baseLTVasCollateral': '0x3c',
 'borrowCumulativeIndex': '0xde0b6b3a7640000',
 'borrowRate': '0x0',
 'borrowingEnabled': '0x1',
 'decimals': '0x12',
 'isActive': '0x1',
 'isFreezed': '0x0',
 'lastUpdateTimestamp': '0x5b30edd821a23',
 'liquidationBonus': '0xa',
 'liquidationThreshold': '0x41',
 'liquidityCumulativeIndex': '0xde0b6b3a7640000',
 'liquidityRate': '0x0',
 'oTokenAddress': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454',
 'reserveAddress': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
 'totalBorrows': '0x0',
 'totalLiquidity': '0x3635c9adc5dea00000',
 'usageAsCollateralEnabled': '0x1'}


In [238]:

# params = {'_address': 'hxc9319d218ffff1672c9f1c15365c6f6c0d161c7b' }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c")\
    .method("getReserves")\
    .build()
response = icon_service.call(_call)
print(response)

['cx072d8f95877a53df350e3dc3d8dba2f379037d42']


In [234]:
# params = {'_reserve': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42','_user':"hx91bf040426f226b3bfcd2f0b5967bbb0320525ce" }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c")\
    .method("getAllReserveData")\
    .build()
    
response = icon_service.call(_call)
pprint(response)

{'USDb': {'availableLiquidity': '0x3635c9adc5dea00000',
          'baseLTVasCollateral': '0x3c',
          'borrowCumulativeIndex': '0xde0b6b3a7640000',
          'borrowRate': '0x0',
          'borrowingEnabled': '0x1',
          'decimals': '0x12',
          'isActive': '0x1',
          'isFreezed': '0x0',
          'lastUpdateTimestamp': '0x5b30edd821a23',
          'liquidationBonus': '0xa',
          'liquidationThreshold': '0x41',
          'liquidityCumulativeIndex': '0xde0b6b3a7640000',
          'liquidityRate': '0x0',
          'oTokenAddress': 'cx48dd26868c25cfa744aab9f2fccd76ed40c41454',
          'reserveAddress': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42',
          'totalBorrows': '0x0',
          'totalLiquidity': '0x3635c9adc5dea00000',
          'usageAsCollateralEnabled': [True]}}


In [165]:

params = {'_reserve': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42'}

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx162fe5b5d3276e9493de1fd440511f0844ca9a7c")\
    .method("getReserveConstants")\
    .params(params)\
    .build()
    
response = icon_service.call(_call)
print(response)

{'optimalUtilizationRate': '0x853a0d2313c0000', 'baseBorrowRate': '0x0', 'slopeRate1': '0xf8b0a10e470000', 'slopeRate2': '0x29a2241af62c0000'}


In [181]:
params = {'_reserve': 'cx072d8f95877a53df350e3dc3d8dba2f379037d42','_availableLiquidity':0,'_totalBorrows':0 }

_call = CallBuilder()\
    .from_(deployer_wallet.get_address())\
    .to("cx84490f03cc04b799fe668545340eaeac7cae628c")\
    .method("calculateInterestRates")\
    .params(params)\
    .build()
    
response = icon_service.call(_call)
print(response)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32003, 'message': 'Method not found: LendingPoolDataProvider.calculateInterestRates'}


JSONRPCException: {'code': -32003, 'message': 'Method not found: LendingPoolDataProvider.calculateInterestRates'} (Json rpc error)